In [7]:
# Cell 1: Import Libraries and Custom Modules
import yfinance as yf
import data_utils
import strategies
import backtest
import grid_search  # Import the new grid search module

# Cell 2: Set Parameters and Prepare Data
# List of top Indonesian stock tickers
tickers = ["TLKM.JK", "BBCA.JK", "BBRI.JK", "UNVR.JK", "ASII.JK"]
start_date = '2024-01-01'

# Define the parameter grids for each strategy
parameter_grids = {
    "LongOnlyMACD": {
        "window_slow": [24, 26, 28],
        "window_fast": [10, 12, 14],
        "window_sign": [8, 9, 10]
    },
    "LongOnlyMomentum": {
        "window": [10, 20, 30]
    },
    "StochasticOscillatorStrategy": {
        "k_window": [10, 14, 20],
        "smooth_k": [2, 3],
        "smooth_d": [2, 3],
        "threshold": [10, 20]
    },
    "StochasticEMAStrategy": {
        "k_length": [80],
        "k_smoothing": [2],
        "d_smoothing": [2],
        "adx_length": [14],
        "adx_smoothing": [14],
        "adx_threshold": [25],
        "rsi_length": [14]
    }
}

# Cell 3: Iterate Over Tickers and Perform Grid Search
overall_best_perf = float('-inf')
overall_best_strategy = None
overall_best_params = None
overall_best_ticker = None

strategy_performance = {name: {'total_perf': 0, 'total_outperf': 0, 'count': 0} for name in parameter_grids.keys()}

for ticker in tickers:
    print(f"\nProcessing ticker: {ticker}")

    # Prepare data using a utility function
    real_data = data_utils.prepare_data(ticker, start_date=start_date)

    if real_data is not None:
        best_results = {}
        
        for strategy_name, param_grid in parameter_grids.items():
            print(f"\nPerforming grid search for {strategy_name} on {ticker}...")

            # Perform grid search for each strategy
            results = grid_search.grid_search(
                strategy_name=strategy_name,
                param_grid=param_grid,
                data=real_data,
                symbol=ticker
            )
            
            # Store the best performance metrics for each strategy
            best_results[strategy_name] = {
                "best_perf": results["best_perf"],
                "best_outperf": results["best_outperf"],
                "best_params_perf": results["best_params_perf"],
                "best_params_outperf": results["best_params_outperf"]
            }

            # Accumulate performance metrics for averaging
            strategy_performance[strategy_name]['total_perf'] += results["best_perf"]
            strategy_performance[strategy_name]['total_outperf'] += results["best_outperf"]
            strategy_performance[strategy_name]['count'] += 1
            
            # Check for overall best performance
            if results["best_perf"] > overall_best_perf:
                overall_best_perf = results["best_perf"]
                overall_best_strategy = strategy_name
                overall_best_params = results["best_params_perf"]
                overall_best_ticker = ticker
            
        # Print the final best perf and outperf for each strategy
        for strategy_name, result in best_results.items():
            print(f"\nStrategy: {strategy_name} on {ticker}")
            print(f"Best Performance (perf): {result['best_perf']} with parameters: {result['best_params_perf']}")
            print(f"Best Outperformance (outperf): {result['best_outperf']} with parameters: {result['best_params_outperf']}")
            
            # Run backtest for the best perf configuration
            print(f"\nRunning best configuration for {strategy_name} based on best perf:")
            perf, outperf, result = backtest.run_backtest(
                strategy_class=strategies.__getattribute__(strategy_name),
                data=real_data,
                symbol=ticker,
                print_results=False,
                **result['best_params_perf']
            )
    else:
        print(f"No data available for backtesting on {ticker}.")

# Cell 4: Print Average Performance and Best Overall Strategy
print("\nAverage Performance and Outperformance for Each Strategy:")
for strategy_name, performance in strategy_performance.items():
    if performance['count'] > 0:
        avg_perf = performance['total_perf'] / performance['count']
        avg_outperf = performance['total_outperf'] / performance['count']
        print(f"\nStrategy: {strategy_name}")
        print(f"Average Performance (perf): {avg_perf}")
        print(f"Average Outperformance (outperf): {avg_outperf}")

print("\nBest Overall Strategy Based on Performance:")
print(f"Strategy: {overall_best_strategy} on Ticker: {overall_best_ticker}")
print(f"Performance: {overall_best_perf} with parameters: {overall_best_params}")


[*********************100%%**********************]  1 of 1 completed


Processing ticker: TLKM.JK

Performing grid search for LongOnlyMACD on TLKM.JK...



Performing grid search for LongOnlyMomentum on TLKM.JK...

Performing grid search for StochasticOscillatorStrategy on TLKM.JK...

Performing grid search for StochasticEMAStrategy on TLKM.JK...

Strategy: LongOnlyMACD on TLKM.JK
Best Performance (perf): 0.9270960744002649 with parameters: {'window_slow': 26, 'window_fast': 12, 'window_sign': 10}
Best Outperformance (outperf): 0.23760952183302753 with parameters: {'window_slow': 26, 'window_fast': 12, 'window_sign': 10}

Running best configuration for LongOnlyMACD based on best perf:

Strategy: LongOnlyMomentum on TLKM.JK
Best Performance (perf): 0.9775399206651584 with parameters: {'window': 20}
Best Outperformance (outperf): 0.2654187085439458 with parameters: {'window': 20}

Running best configuration for LongOnlyMomentum based on best perf:

Strategy: StochasticOscillatorStrategy on TLKM.JK
Best Performance (perf): 1.0456373985364054 with parameters: {'k_window': 20, 'smooth_k': 2, 'smooth_d': 3, 'threshold': 10}
Best Outperformance

[*********************100%%**********************]  1 of 1 completed


Strategy: StochasticEMAStrategy on TLKM.JK
Best Performance (perf): 0 with parameters: {'k_length': 80, 'k_smoothing': 2, 'd_smoothing': 2, 'adx_length': 14, 'adx_smoothing': 14, 'adx_threshold': 25, 'rsi_length': 14}
Best Outperformance (outperf): 0 with parameters: {'k_length': 80, 'k_smoothing': 2, 'd_smoothing': 2, 'adx_length': 14, 'adx_smoothing': 14, 'adx_threshold': 25, 'rsi_length': 14}

Running best configuration for StochasticEMAStrategy based on best perf:

Processing ticker: BBCA.JK

Performing grid search for LongOnlyMACD on BBCA.JK...


...

[*********************100%%**********************]  1 of 1 completed


Strategy: StochasticEMAStrategy on UNVR.JK
Best Performance (perf): 0 with parameters: {'k_length': 80, 'k_smoothing': 2, 'd_smoothing': 2, 'adx_length': 14, 'adx_smoothing': 14, 'adx_threshold': 25, 'rsi_length': 14}
Best Outperformance (outperf): 0 with parameters: {'k_length': 80, 'k_smoothing': 2, 'd_smoothing': 2, 'adx_length': 14, 'adx_smoothing': 14, 'adx_threshold': 25, 'rsi_length': 14}

Running best configuration for StochasticEMAStrategy based on best perf:

Processing ticker: ASII.JK

Performing grid search for LongOnlyMACD on ASII.JK...



Performing grid search for LongOnlyMomentum on ASII.JK...

Performing grid search for StochasticOscillatorStrategy on ASII.JK...


C:\Users\Intel X Nvidia RTX\anaconda3\envs\py10\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning:

Mean of empty slice.

C:\Users\Intel X Nvidia RTX\anaconda3\envs\py10\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning:

Mean of empty slice.

C:\Users\Intel X Nvidia RTX\anaconda3\envs\py10\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning:

Mean of empty slice.

C:\Users\Intel X Nvidia RTX\anaconda3\envs\py10\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning:

Mean of empty slice.




Performing grid search for StochasticEMAStrategy on ASII.JK...

Strategy: LongOnlyMACD on ASII.JK
Best Performance (perf): 0.8913879573759799 with parameters: {'window_slow': 28, 'window_fast': 12, 'window_sign': 9}
Best Outperformance (outperf): 0.0038496715722959873 with parameters: {'window_slow': 26, 'window_fast': 12, 'window_sign': 10}

Running best configuration for LongOnlyMACD based on best perf:

Strategy: LongOnlyMomentum on ASII.JK
Best Performance (perf): 0.9746751883822495 with parameters: {'window': 30}
Best Outperformance (outperf): 0.1478894740965354 with parameters: {'window': 30}

Running best configuration for LongOnlyMomentum based on best perf:

Strategy: StochasticOscillatorStrategy on ASII.JK
Best Performance (perf): 1.084633103220499 with parameters: {'k_window': 20, 'smooth_k': 3, 'smooth_d': 2, 'threshold': 20}
Best Outperformance (outperf): 0.18121846907415773 with parameters: {'k_window': 20, 'smooth_k': 3, 'smooth_d': 2, 'threshold': 20}

Running best con

In [2]:
backtest

<module 'backtest' from 'C:\\Users\\Intel X Nvidia RTX\\ZenTraderML\\ZenTraderML-\\backtest.py'>